<a href="https://colab.research.google.com/github/alessandronicolini/IncrementalLearning/blob/main/LearningWithoutForgetting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# upload work files from your git hub repository
import sys

!git clone https://github.com/alessandronicolini/IncrementalLearning.git # clone proj repository
!rm -rf IncrementalLearning/README.md 
!rm -rf IncrementalLearning/baselines.ipynb

path = 'IncrementalLearning/'
if path not in sys.path:
    sys.path.append('IncrementalLearning/')

!pip3 install import_ipynb

fatal: destination path 'IncrementalLearning' already exists and is not an empty directory.


In [2]:
import torch
import os
import torch.nn as nn
from torch.utils.data import Subset, DataLoader
import torch.optim as optim
from torchvision import transforms
import numpy as np
import import_ipynb
import copy
# project classes --------------------------------------------------------------
from cifar100 import ilCIFAR100
from resnet_cifar import resnet32

importing Jupyter notebook from cifar100.ipynb
Files already downloaded and verified
[60, 34, 84, 67, 85, 44, 18, 48, 1, 47, 61, 35, 82, 58, 76, 29, 71, 0, 79, 93, 56, 90, 20, 43, 26, 7, 73, 25, 9, 65, 95, 51, 11, 2, 74, 28, 96, 27, 99, 64, 70, 42, 62, 8, 98, 77, 39, 88, 10, 94, 3, 52, 68, 32, 5, 72, 38, 75, 69, 30, 40, 41, 24, 55, 91, 45, 12, 16, 22, 53, 63, 57, 31, 33, 21, 83, 49, 81, 59, 78, 97, 19, 46, 17, 36, 87, 6, 13, 14, 89, 80, 23, 86, 37, 54, 92, 50, 66, 15, 4]
0
1
2
3
4
5
6
7
8
9
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [5]:
class lwf(nn.Module):
  def __init__(self,network):
    super(lwf, self).__init__()
    self.model=network.to('cuda')
    self.batch_size=128
    self.CLASSES_PER_BATCH=10
    self.NUM_BATCHES=10
    self.LR=2
    self.MOMENTUM=0.9
    self.WEIGHT_DECAY=1e-5
    self.MILESTONES=[49,63]
    self.GAMMA=0.2
    self.numepochs=70
    self.randomseed=10
    self.trainloader=None
    self.testloader=None
    self.original_training_set = ilCIFAR100(self.CLASSES_PER_BATCH,self.randomseed)
    self.original_test_set = ilCIFAR100(self.CLASSES_PER_BATCH,self.randomseed, train=False)
    self.criterion=nn.BCEWithLogitsLoss()
    self.optimizer = optim.SGD(self.model.parameters(), lr=self.LR, momentum=self.MOMENTUM, weight_decay=self.WEIGHT_DECAY)
    self.scheduler = optim.lr_scheduler.MultiStepLR(self.optimizer, milestones=self.MILESTONES, gamma=self.GAMMA)
    self.firstbatch=1
    self.classes_seen=0
    self.diz = self.original_training_set.get_dict()
  def forward(self, x):
    self.model = self.model.cuda()
    return self.model.forward(x)
  def update_parameters(self):
    old_model = copy.deepcopy(self)
    old_model.eval()
    old_model.to('cuda')
    for epoch in range(self.numepochs):
      #print(epoch)
        
        for inputs,labels in self.trainloader:
          labels = torch.tensor([torch.tensor(self.diz[c.item()]) for c in labels])
          inputs=inputs.to('cuda')
          labels=labels.to('cuda')
          self.optimizer.zero_grad()
          outputs=self.model(inputs)

          labels_encoded=torch.eye(100)[labels] #CAMBIARE ONE_HOT
          labels_encoded=labels_encoded.to('cuda')
          if self.firstbatch:
            loss=self.criterion(outputs[:,:self.batch_size],labels_encoded)
          else:
            old_target=old_model(inputs)
            old_target=torch.sigmoid(old_target).cuda()
            target = torch.cat((old_target[:,:self.classes_seen],labels_encoded [:, self.classes_seen:self.classes_seen+self.batch_size]), dim=1)
            loss=self.criterion(outputs[:,:self.classes_seen+self.batch_size],target)
          loss.backward()
          self.optimizer.step()
        self.scheduler.step()

        
              # deep copy the model

  def training_model(self):
    
    train_indices = self.original_training_set.get_batch_indexes()
    test_indices = self.original_test_set.get_batch_indexes()
    current_test_indexes=[]
    
    
    acc=[]
    accuracy=0
    for i in range(self.NUM_BATCHES):
  
      train_dataset=Subset(self.original_training_set,train_indices[i])
      current_test_indexes+=test_indices[i].tolist()
      test_dataset=Subset(self.original_test_set,current_test_indexes)
      self.trainloader = DataLoader(train_dataset, batch_size=self.batch_size, shuffle=True)
      self.testloader = DataLoader(test_dataset, batch_size=self.batch_size, shuffle=True)        
      self.train()
      self.update_parameters()
      print('si porco dio')
      self.firstbatch=0
      self.classes_seen+=10
      self.eval() # Set Network to evaluation mode
      running_corrects = 0
      y_true = []
      y_pred = []
      for inputs, labels in self.testloader:
          labels = torch.tensor([torch.tensor(self.diz[c.item()]) for c in labels])
          inputs = inputs.to('cuda')
          labels = labels.to('cuda')
          outputs = self.model(inputs)
          _, preds = torch.max(outputs.data, 1)
          running_corrects += torch.sum(preds == labels.data).data.item()
          labels=labels.detach().cpu().numpy()
          labels=labels.tolist()
          y_true.extend(labels)
          preds=preds.detach().cpu().numpy()
          y_pred_tmp = [p for p in preds]
          y_pred.extend(y_pred_tmp)
      accuracy = running_corrects / float(len(self.testloader.dataset))
      print('Test Accuracy: %.2f' % (100.0 * accuracy))
      print('-' * 80)
      acc.append(accuracy)
    return self
    

In [6]:
model=lwf(network=resnet32(num_classes=100)).training_model()


Files already downloaded and verified
[73, 4, 54, 61, 99, 1, 26, 59, 62, 35, 83, 20, 98, 66, 91, 41, 9, 31, 46, 5, 53, 17, 77, 45, 48, 79, 36, 33, 58, 22, 38, 81, 78, 71, 30, 56, 75, 2, 37, 0, 15, 8, 12, 19, 34, 23, 49, 92, 88, 42, 90, 28, 27, 65, 87, 84, 96, 51, 32, 10, 14, 93, 52, 85, 95, 82, 64, 76, 80, 44, 57, 18, 11, 72, 68, 3, 24, 39, 55, 6, 47, 13, 63, 70, 7, 40, 25, 74, 89, 86, 29, 94, 67, 97, 69, 16, 43, 60, 50, 21]
Files already downloaded and verified
[73, 4, 54, 61, 99, 1, 26, 59, 62, 35, 83, 20, 98, 66, 91, 41, 9, 31, 46, 5, 53, 17, 77, 45, 48, 79, 36, 33, 58, 22, 38, 81, 78, 71, 30, 56, 75, 2, 37, 0, 15, 8, 12, 19, 34, 23, 49, 92, 88, 42, 90, 28, 27, 65, 87, 84, 96, 51, 32, 10, 14, 93, 52, 85, 95, 82, 64, 76, 80, 44, 57, 18, 11, 72, 68, 3, 24, 39, 55, 6, 47, 13, 63, 70, 7, 40, 25, 74, 89, 86, 29, 94, 67, 97, 69, 16, 43, 60, 50, 21]
si porco dio
Test Accuracy: 83.00
--------------------------------------------------------------------------------
si porco dio
Test Accuracy:

In [24]:
<trainmodel=model.training()

TypeError: ignored